## Process
This code splits the dataset into training, validate, and test sets. 

In [2]:
import os
import shutil
import random
from pathlib import Path

# Define source folders
source_folders = {
    "cat": "cropped_images/cat_images",
    "dog": "cropped_images/dog_images",
    "human": "human_images"
}

# Define target base folder
target_base = "dataset"
splits = ["train", "val", "test"]
split_ratios = [0.8, 0.1, 0.1]

# Create directory structure
for split in splits:
    for label in source_folders.keys():
        Path(f"{target_base}/{split}/{label}").mkdir(parents=True, exist_ok=True)

# Process each class folder
for label, folder in source_folders.items():
    images = os.listdir(folder)
    random.shuffle(images)

    total = len(images)
    train_end = int(split_ratios[0] * total)
    val_end = train_end + int(split_ratios[1] * total)

    split_data = {
        "train": images[:train_end],
        "val": images[train_end:val_end],
        "test": images[val_end:]
    }

    for split, files in split_data.items():
        for file in files:
            src = os.path.join(folder, file)
            dst = os.path.join(target_base, split, label, file)
            shutil.copy2(src, dst)

print("✅ Dataset successfully split into train, val, and test folders.")


✅ Dataset successfully split into train, val, and test folders.


# Dealing with Human Images
This code picks 2500 random human images which can be used in training. The images that were there were too many and could have biased the model.  

In [4]:
import os
import random
import shutil

# Paths
original_human_path = 'dataset/train/human'
output_path = 'working_dataset/balanced_train/_human'
os.makedirs(output_path, exist_ok=True)

# Number of human images to retain
target_human_count = 2500

# Get list of human image files
human_images = [f for f in os.listdir(original_human_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Randomly select 2500 images
selected_images = random.sample(human_images, target_human_count)

# Copy selected images to new folder
for fname in selected_images:
    src = os.path.join(original_human_path, fname)
    dst = os.path.join(output_path, fname)
    shutil.copyfile(src, dst)

print(f"✅ Copied {len(selected_images)} human images to {output_path}")


✅ Copied 2500 human images to working_dataset/balanced_train/_human


# Augment the images
This code augments an imbalanced dataset to ensure each class (cat, dog, human) reaches a specific number of images. It first copies the original images to a new output directory and then generates additional augmented images using random transformations until the desired count per class is met. This helps balance the dataset, improving model fairness and performance across classes.

In [1]:
import os
import random
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

input_path = 'working_dataset/balanced_train'
output_path = 'augmented_dataset_balanced'
os.makedirs(output_path, exist_ok=True)

target_counts = {
    'cat': 3000,
    'dog': 3000,
    'human': 3000
}

def get_transform(class_name):
    if class_name == 'cat':
        return transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(degrees=(0, 270)),  # Covers 90°, 180°, 270°
            transforms.ColorJitter(
                brightness=random.uniform(0.3, 0.8),
                contrast=random.uniform(0.3, 0.8),
                saturation=random.uniform(0.3, 0.8)
            ),
            transforms.RandomAffine(degrees=30, translate=(0.1, 0.1)),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])
    else:
        return transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(
                brightness=random.uniform(0.2, 0.4),
                contrast=random.uniform(0.2, 0.4)
            ),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

to_pil = transforms.ToPILImage()

for class_name in target_counts:
    src_folder = os.path.join(input_path, class_name)
    dst_folder = os.path.join(output_path, class_name)
    os.makedirs(dst_folder, exist_ok=True)

    image_files = [f for f in os.listdir(src_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for fname in image_files:
        Image.open(os.path.join(src_folder, fname)).save(os.path.join(dst_folder, fname))

    current_count = len(image_files)
    needed = target_counts[class_name] - current_count

    print(f"\n🔄 Augmenting {class_name.upper()}... ({current_count} → {target_counts[class_name]})")

    transform = get_transform(class_name)

    for i in tqdm(range(needed)):
        img_name = random.choice(image_files)
        img_path = os.path.join(src_folder, img_name)

        try:
            image = Image.open(img_path).convert("RGB")
            transformed_tensor = transform(image)
            transformed_image = to_pil(transformed_tensor)
            new_name = f"{os.path.splitext(img_name)[0]}_aug{i}.jpg"
            transformed_image.save(os.path.join(dst_folder, new_name))
        except Exception as e:
            print(f"⚠️ Error processing {img_name}: {e}")




🔄 Augmenting CAT... (911 → 3000)


100%|███████████████████████████████████████| 2089/2089 [01:03<00:00, 32.65it/s]



🔄 Augmenting DOG... (2625 → 3000)


100%|█████████████████████████████████████████| 375/375 [00:13<00:00, 27.73it/s]



🔄 Augmenting HUMAN... (2500 → 3000)


100%|█████████████████████████████████████████| 500/500 [00:12<00:00, 40.13it/s]


In [5]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import os

# Define transforms again if needed
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Set new base path (update as necessary)
base_path = 'cleaned_and_ready_for_use_data' 

# Recreate datasets
train_dataset = ImageFolder(root=os.path.join(base_path, 'train'), transform=transform)
val_dataset = ImageFolder(root=os.path.join(base_path, 'val'), transform=transform)
test_dataset = ImageFolder(root=os.path.join(base_path, 'test'), transform=transform)

# Recreate data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("done")

print(f"Train: {len(train_dataset)}")
print(f"Val:   {len(val_dataset)}")
print(f"Test:  {len(test_dataset)}")


done
Train: 9000
Val:   1764
Test:  1768
